# Reading waterinfo observations

This notebook introduces how to use the `hydropandas` package to read, process and visualise data from the Waterinfo database. In this notebook the `ddlpy` package is used to access the waterinfo api. This package can be installed using `pip install ddlpy`.

In [ ]:

import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import nlmod
import datetime as dt
import hydropandas as hpd
from hydropandas.io.waterinfo import get_locations_gdf

# enabling debug logging so we can see what happens in the background
hpd.util.get_color_logger("INFO");

In [ ]:
# settings
grootheid_code = None
locatie = 'SCHOONHVN'
tmin = dt.datetime(2024, 1, 1)
tmax = dt.datetime(2024, 1, 3)
extent = (110000, 125000, 429550, 449900) # Schoonhoven

In [ ]:
# get waterinfo observations within an extent
oc = hpd.read_waterinfo(
    extent=extent, grootheid_code=grootheid_code, locatie=locatie, tmin=tmin, tmax=tmax
)
oc

In [ ]:
# show all measurement types within the extent
gdf_meas = get_locations_gdf(extent=extent)
gdf_meas[['Grootheid.Code','Grootheid.Omschrijving']]

In [ ]:
# get data from a certain location and grootheid
o = hpd.WaterlvlObs.from_waterinfo(
    locatie='SCHOONHVN',
    grootheid_code='WATHTE',
    tmin=tmin,
    tmax=tmax,
    location_gdf=gdf_meas, #specifying the location_gdf signficantly speeds up the process
)
o.plot(ylabel=o.unit, title=o.name, legend=False)

In [ ]:
# get data from a certain location and grootheid
o = hpd.WaterlvlObs.from_waterinfo(
    locatie='SCHOONHVN',
    grootheid_code='WATHTBRKD',
    tmin=tmin,
    tmax=tmax,
    location_gdf=gdf_meas, #specifying the location_gdf signficantly speeds up the process
)
o.plot(ylabel=o.unit, title=o.name, legend=False)

In [ ]:
# get all measurement points within the Netherlands
gdf = hpd.io.waterinfo.get_locations_gdf()
ax = gdf.plot(figsize=(10,10))
nlmod.plot.add_background_map(ax=ax)

## Ophalen data WaterWebservices <a id=GroundwaterObs></a>

In de toekomst komen de WaterWebservices beschikbaar. Deze werken op dit moment (2024-2-26) nog niet. Bij het ophalen van de metingen wordt altijd aangegeven dat het maximaal aantal metingen wordt overschreven. Zelfs als je maar voor één dag metingen ophaalt.

Handige links:
- https://waterwebservices.beta.rijkswaterstaat.nl/test/swagger-ui/index.html#/
- https://rijkswaterstaatdata.nl/projecten/beta-waterwebservices/


In [ ]:
import requests
import json
import pandas as pd
from shapely.geometry import Point

In [ ]:
# request catalogus REST API
url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/METADATASERVICES/OphalenCatalogus"
body = {"CatalogusFilter": {"Compartimenten":True, "Grootheden":True}}
headers = {"content-type": "application/json"}
r = requests.post(url,  data=json.dumps(body), headers=headers)

out = r.json()

In [ ]:
# plot locations from REST API
geometries = [Point(loc['Lon'], loc['Lat']) for loc in out['LocatieLijst']]
gdf = gpd.GeoDataFrame(out['LocatieLijst'], geometry=geometries, crs=4258)
gdf.to_crs(28992, inplace=True)
extent_nl_poly = nlmod.util.polygon_from_extent([482.06, 306602.42, 284182.97, 637049.52])
gdf = gdf.loc[gdf.within(extent_nl_poly)]
ax = gdf.plot(figsize=(10,10))
nlmod.plot.add_background_map(ax=ax)

In [ ]:
# read REST API using an extent (Schoonhoven zuid-west)
extent_schoon = nlmod.util.polygon_from_extent((117850, 118180, 439550, 439900))
gdf_schoon = gdf.loc[gdf.within(extent_schoon.buffer(10000))]
ax = gdf_schoon.plot('Code',figsize=(10,10), legend=True)
nlmod.plot.add_background_map(ax=ax, alpha=0.5)

In [ ]:
# kies code en laat zien welke parameters erbij horen
code = 'schoonhoven'

df_meta_locatie = pd.DataFrame(out['AquoMetadataLocatieLijst']).set_index('Locatie_MessageID')
df_meta = pd.DataFrame(out['AquoMetadataLijst'])

locatie_message_id = gdf.loc[gdf['Code']==code,'Locatie_MessageID'].iloc[0]

AquoMetaData_MessageIDs = df_meta_locatie.loc[locatie_message_id, 'AquoMetaData_MessageID']
if isinstance(AquoMetaData_MessageIDs, int):
    AquoMetaData_MessageIDs = [AquoMetaData_MessageIDs]
df_meta.loc[AquoMetaData_MessageIDs]


In [ ]:
# request om metingen op te halen
code = 'ameland.nes'
aquometadata_message_id = 6

url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/ONLINEWAARNEMINGENSERVICES/OphalenWaarnemingen"
body = {"Locatie": {"Code": code},
        "AquoPlusWaarnemingMetadata":  {"AquoMetadata":{"Compartiment":{"Code":df_meta.loc[aquometadata_message_id,'Compartiment']['Code']}, 
                                                        "Grootheid":{"Code":df_meta.loc[aquometadata_message_id,'Grootheid']['Code']}}},
        "Periode":{"Begindatumtijd":"2024-06-01T00:00:00.000+01:00",
                   "Einddatumtijd":"2025-01-01T00:00:00.000+01:00"}}
headers = {"content-type": "application/json"}
r = requests.post(url,  data=json.dumps(body), headers=headers)
if r.status_code==204:
    print('No data available')
else:
    meas = r.json()
    print(meas)

In [ ]:
# andere poging
code = 'ameland.nes'
aquometadata_message_id = 6

url = "https://waterwebservices.beta.rijkswaterstaat.nl/test/ONLINEWAARNEMINGENSERVICES/OphalenWaarnemingen"
body = {"Locatie": {"Code": code},
        "AquoPlusWaarnemingMetadata":  {"AquoMetadata":{"ProcesType":"verwachting", 
                                                        "Grootheid":{"Code":df_meta.loc[aquometadata_message_id,'Grootheid']['Code']}}},
        "Periode":{"Begindatumtijd":"2024-01-01T00:00:00.000+01:00",
                   "Einddatumtijd":"2024-01-02T00:00:00.000+01:00"}}
headers = {"content-type": "application/json"}
r = requests.post(url,  data=json.dumps(body), headers=headers)
if r.status_code==204:
    print('No data available')
else:
    meas = r.json()
    print(meas)